In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, date
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit

import argparse

/Users/hpnhxxwn/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [90]:
print('Loading raw data...')
train_users_path='input/train_users_2.csv'
test_users_path='input/test_users.csv'
sessions_path='input/sessions.csv'

#Note: age_gender_bkts.csv and countries.csv files are not used.

#########Loading data#############
#train_users
df_train = pd.read_csv(train_users_path)
target = df_train['country_destination']
#df_train = df_train.drop(['country_destination'], axis=1)

#test_users
df_test = pd.read_csv(test_users_path)    
id_test = df_test['id']

#sessions
df_sessions = pd.read_csv(sessions_path)
df_sessions['id'] = df_sessions['user_id']
df_sessions = df_sessions.drop(['user_id'],axis=1)

Loading raw data...


In [3]:
#df_train.date_account_created

In [4]:
df_train.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [91]:
df_train.gender.replace("-unknown-", np.nan, inplace=True)
#df_train.head()
df_test.gender.replace("-unknown-", np.nan, inplace=True)


In [92]:
pd.unique(df_train.gender.values)

array([nan, 'MALE', 'FEMALE', 'OTHER'], dtype=object)

In [93]:
#sessions grouping by user

#Group by user_id, aggregate by number of counts (counting device_type as it is never NA), 
#and total sum of elapsed time in seconds
group_sessions = df_sessions.groupby("id")
#.agg({'device_type':'count', 'secs_elapsed':'sum'})
#rename columns
# // group_sessions.columns = ['sum_secs_elapsed', 'counts']
#group by variable turns into index, I'm reseting the index and putting user_id back as a column
# //group_sessions.reset_index(level=0, inplace=True)

In [94]:
group_sessions.head()

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NaN,NaN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NaN,NaN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NaN,NaN,Windows Desktop,435.0,d1mm9tcy42
127,dashboard,view,dashboard,Mac Desktop,2739.0,yo8nz8bqcq
128,create,submit,create_user,Mac Desktop,NaN,yo8nz8bqcq
129,confirm_email,click,confirm_email_link,Mac Desktop,115983.0,yo8nz8bqcq
130,show,view,p3,Mac Desktop,20285.0,yo8nz8bqcq
131,show_personalize,data,user_profile_content_update,Mac Desktop,3255.0,yo8nz8bqcq


In [95]:
group_sessions = group_sessions.agg({'device_type':'count', 'secs_elapsed':'sum'})

In [96]:
group_sessions.head()

,device_type,secs_elapsed
id,,
00023iyk9l,40,867896.0
0010k6l0om,63,586543.0
001wyh0pz8,90,282965.0
0028jgx1x1,31,297010.0
002qnbzfs5,789,6487080.0


In [97]:
group_sessions.columns = ['sum_secs_elapsed', 'counts']

In [98]:
group_sessions.head()

,sum_secs_elapsed,counts
id,,
00023iyk9l,40,867896.0
0010k6l0om,63,586543.0
001wyh0pz8,90,282965.0
0028jgx1x1,31,297010.0
002qnbzfs5,789,6487080.0


In [99]:
group_sessions.reset_index(level=0, inplace=True)
group_sessions.head()

,id,sum_secs_elapsed,counts
0,00023iyk9l,40,867896.0
1,0010k6l0om,63,586543.0
2,001wyh0pz8,90,282965.0
3,0028jgx1x1,31,297010.0
4,002qnbzfs5,789,6487080.0


In [100]:
df_train.age[0:20]

0      NaN
1     38.0
2     56.0
3     42.0
4     41.0
5      NaN
6     46.0
7     47.0
8     50.0
9     46.0
10    36.0
11    47.0
12     NaN
13    37.0
14    36.0
15    33.0
16     NaN
17    31.0
18     NaN
19    29.0
Name: age, dtype: float64

In [101]:
df_train.age.replace(np.nan, 0, inplace=True)
df_train.age.astype(str, inplace=True)
df_train.head()
#df_train["age_bucket"] = ""
#df_train.age_bucket

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,0.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [102]:
#bucket all ages into format that user_demo is in for age
import math
def agebuckets(ages):
    ageless =  [i for i in range(5,101,5)] # 5, 10, 15, 20...95, 100
    buckets = ['%d-%d' %(i, i+4) for i in range(0,100,5)] # 0-4, 5-9, 10-14...90-94, 95-99
    newlist = []
    for i in range(len(ages)):
        if math.isnan(ages[i]):
            newlist.append('NA')
        elif ages[i] <ageless[0]:
            newlist.append(buckets[0])
        elif ages[i] < ageless[1]:
            newlist.append(buckets[1])
        elif ages[i] < ageless[2]:
            newlist.append(buckets[2])
        elif ages[i] < ageless[3]:
            newlist.append(buckets[3])
        elif ages[i] < ageless[4]:
            newlist.append(buckets[4])
        elif ages[i] < ageless[5]:
            newlist.append(buckets[5])
        elif ages[i] < ageless[6]:
            newlist.append(buckets[6])
        elif ages[i] < ageless[7]:
            newlist.append(buckets[7])
        elif ages[i] < ageless[8]:
            newlist.append(buckets[8])
        elif ages[i] < ageless[9]:
            newlist.append(buckets[9])
        elif ages[i] < ageless[10]:
            newlist.append(buckets[10])
        elif ages[i] < ageless[11]:
            newlist.append(buckets[11])
        elif ages[i] < ageless[12]:
            newlist.append(buckets[12]) 
        elif ages[i] < ageless[13]:
            newlist.append(buckets[13]) 
        elif ages[i] < ageless[14]:
            newlist.append(buckets[14])
        elif ages[i] < ageless[15]:
            newlist.append(buckets[15])
        elif ages[i] < ageless[16]:
            newlist.append(buckets[16])
        elif ages[i] < ageless[17]:
            newlist.append(buckets[17])
        elif ages[i] < ageless[18]:
            newlist.append(buckets[18])
        elif ages[i] < ageless[19]:
            newlist.append(buckets[19]) 
        else:
            newlist.append('100+')
    return newlist

In [103]:
df_train.age = agebuckets(df_train.age)
df_test.age = agebuckets(df_test.age)

In [104]:
df_train.date_account_created = pd.to_datetime(df_train.date_account_created)
df_train.timestamp_first_active = pd.to_datetime(df_train.timestamp_first_active, format = "%Y%m%d%H%M%S")
df_train.date_first_booking = pd.to_datetime(df_train.date_first_booking)
df_test.timestamp_first_active = pd.to_datetime(df_test.timestamp_first_active, format = "%Y%m%d%H%M%S")
df_test.date_account_created = pd.to_datetime(df_test.date_account_created)

In [105]:
def timedif(L1, L2):
    timediflist = []
    for i in range(len(L1)):
        try:
            if (L1[i]-L2[i]).days < 0:#datetime.timedelta(days=0):
                timediflist.append('before')
            elif (L1[i]-L2[i]).days ==0: #datetime.timedelta(days=1):
                timediflist.append('same day')
            else:
                timediflist.append('greater 1 day')
        except:
            timediflist.append('NB')
            
    return timediflist
    

In [106]:
#df_train.date_first_booking - df_train.date_account_created

In [112]:
#adding time lag columns
df_train['first_book_lag'] = timedif(df_train.date_first_booking, df_train.date_account_created)
df_train['account_active_lag'] = timedif(df_train.date_first_booking, df_train.timestamp_first_active)
df_train['account_created_lag'] = timedif(df_train.date_account_created, df_train.timestamp_first_active)

In [136]:
df_test['account_created_lag'] = timedif(df_test.date_account_created, df_test.timestamp_first_active)
df_test['first_book_lag'] = timedif(df_test.date_first_booking, df_test.date_account_created)
df_test['account_active_lag'] = timedif(df_test.date_first_booking, df_test.timestamp_first_active)

In [137]:
df_test['account_active_lag']

0        NB
1        NB
2        NB
3        NB
4        NB
5        NB
6        NB
7        NB
8        NB
9        NB
10       NB
11       NB
12       NB
13       NB
14       NB
15       NB
16       NB
17       NB
18       NB
19       NB
20       NB
21       NB
22       NB
23       NB
24       NB
25       NB
26       NB
27       NB
28       NB
29       NB
         ..
62066    NB
62067    NB
62068    NB
62069    NB
62070    NB
62071    NB
62072    NB
62073    NB
62074    NB
62075    NB
62076    NB
62077    NB
62078    NB
62079    NB
62080    NB
62081    NB
62082    NB
62083    NB
62084    NB
62085    NB
62086    NB
62087    NB
62088    NB
62089    NB
62090    NB
62091    NB
62092    NB
62093    NB
62094    NB
62095    NB
Name: account_active_lag, dtype: object

In [114]:
def bookings(L1, L2, L3, L4):
    timediflist = []
    for i in range(len(L1)):
        if L1[i] == 'same day' or L2[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'same day':
            timediflist.append('early')
        elif L1[i] == 'greater 1 day' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'greater 1 day' and L2[i] == 'before':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'greater 1 day':
            timediflist.append('waited')
        elif L1[i] == 'before' and L2[i] == 'before' and L3[i] == 'greater 1 day':
            timediflist.append('waited')
        elif (len(L4) > 0 ) and L4[i] == 'NDF':
            timediflist.append('NB')
        else:
            timediflist.append('NA')

            
    return timediflist

In [115]:
booking = bookings(df_train.first_book_lag, df_train.account_active_lag, df_train.account_created_lag, target)
test_booking = bookings(df_test.first_book_lag, df_test.account_active_lag, df_test.account_created_lag, [])

In [117]:
df_train['bookings'] = booking


In [149]:
df_test["bookings"] = test_booking

In [138]:
#df_test.drop(["account_created_lag", "first_book_lag", "account_active_lag", "bookings"], axis=1, inplace=True)

In [127]:
#given the train data gender, age, and country_desination produce the corresponding population in thousands
countries = pd.read_csv('input/countries.csv')
user_demo = pd.read_csv('input/age_gender_bkts.csv')
df_train.gender = df_train.gender.astype(str)
population_in_thous = []
for i in range(df_train.shape[0]):
    #print(type(df_train.loc[i, "gender"]))
    if target[i] == 'NDF':
        population_in_thous.append('NB')    
    elif df_train.gender[i] == 'NA' or df_train.age[i] == 'NA' or df_train.loc[i, "gender"].lower() == 'nan': 
        population_in_thous.append('NA')
    elif df_train.gender[i] == 'OTHER':
        population_in_thous.append(0)  
    elif target[i] == 'other':
        gendersi = user_demo.loc[user_demo.gender == df_train.loc[i, "gender"].lower(),:] 
        ages = gendersi.loc[gendersi.age_bucket == df_train.age[i], :]
        ages = list(map(lambda x: float(x), ages.population_in_thousands))
        population_in_thous.append(np.mean(ages))
    else:
        #print(df_train.loc[i, "gender"].lower())
        genders = user_demo.loc[user_demo.gender == df_train.loc[i, "gender"].lower(),:] 
        dests = genders.loc[genders.country_destination == df_train.country_destination[i] ,:]    
        #print ((dests.loc[dests.age_bucket == df_train.age[i], 'population_in_thousands']))
        population_in_thous.append(float((dests.loc[dests.age_bucket == df_train.age[i], 'population_in_thousands'])))
    #break
population_in_thous[0:10]

['NB',
 'NB',
 11264.0,
 2458.8000000000002,
 'NA',
 'NA',
 10659.0,
 10659.0,
 11413.0,
 'NA']

In [128]:
#df_train.gender == "NB"

In [129]:
#merging gender age bucket with train data
df_train['population_in_thousands'] = population_in_thous

In [ ]:
train_m = pd.merge(df_train, group_sessions, left_on='id', right_on ='id', how='left')
train_m = train_m.drop('id', 1)

In [150]:
#merging with grouped sessions and countries, **note most of training data is not in sessions. see below 
test_m = pd.merge(df_test, group_sessions, left_on='id', right_on ='id', how='left')
test_m = test_m.drop('id', 1)

#print (train_m.iloc[0:5, 0:10]) #to be deleted?
#print (train_m.iloc[0:5, 10:])  # to be deleted?

In [131]:
train_m.head()

,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,first_device_type,first_browser,country_destination,first_book_lag,account_active_lag,account_created_lag,bookings,population_in_thousands,sum_secs_elapsed,counts
0,2010-06-28,2009-03-19 04:32:55,NaT,nan,0-4,facebook,0,en,direct,direct,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
1,2011-05-25,2009-05-23 17:48:09,NaT,MALE,35-39,facebook,0,en,seo,google,...,Mac Desktop,Chrome,NDF,NB,NB,greater 1 day,NB,NB,NaN,NaN
2,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,55-59,basic,3,en,direct,direct,...,Windows Desktop,IE,US,before,greater 1 day,greater 1 day,waited,11264,NaN,NaN
3,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,40-44,facebook,0,en,direct,direct,...,Mac Desktop,Firefox,other,greater 1 day,greater 1 day,greater 1 day,waited,2458.8,NaN,NaN
4,2010-09-14,2009-12-08 06:11:05,2010-02-18,nan,40-44,basic,0,en,direct,direct,...,Mac Desktop,Chrome,US,before,greater 1 day,greater 1 day,waited,NA,NaN,NaN


In [151]:
toremove = ['date_account_created', 'timestamp_first_active', 'date_first_booking', 
              'country_destination'] 
train_m.drop(toremove, axis=1, inplace=True)

ValueError: labels ['date_account_created' 'timestamp_first_active' 'date_first_booking'
 'country_destination'] not contained in axis

In [152]:
toremove = ['date_account_created', 'timestamp_first_active', 'date_first_booking'] 
test_m.drop(toremove, axis=1, inplace=True)

In [153]:
print(train_m.head())

   gender    age signup_method  signup_flow language affiliate_channel  \
0     nan    0-4      facebook            0       en            direct   
1    MALE  35-39      facebook            0       en               seo   
2  FEMALE  55-59         basic            3       en            direct   
3  FEMALE  40-44      facebook            0       en            direct   
4     nan  40-44         basic            0       en            direct   

  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0             direct               untracked        Web       Mac Desktop   
1             google               untracked        Web       Mac Desktop   
2             direct               untracked        Web   Windows Desktop   
3             direct               untracked        Web       Mac Desktop   
4             direct               untracked        Web       Mac Desktop   

  first_browser first_book_lag account_active_lag account_created_lag  \
0        Chrome    

In [154]:
print(test_m.head())

   gender    age signup_method  signup_flow language affiliate_channel  \
0  FEMALE  35-39      facebook            0       en            direct   
1     NaN     NA         basic            0       en            direct   
2     NaN     NA         basic            0       en            direct   
3     NaN     NA         basic            0       en            direct   
4     NaN     NA         basic            0       en            direct   

  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0             direct               untracked      Moweb            iPhone   
1             direct               untracked      Moweb            iPhone   
2             direct                  linked        Web   Windows Desktop   
3             direct                  linked        Web   Windows Desktop   
4             direct               untracked        Web       Mac Desktop   

   first_browser account_created_lag first_book_lag account_active_lag  \
0  Mobile Safari  

In [58]:
from sklearn.feature_extraction import DictVectorizer
train_dict = train_m.T.to_dict()
test_dict = test_m.T.to_dict()


In [59]:
#pd.DataFrame.from_dict(train_dict)

In [155]:
train_m = train_m.T.to_dict().values()
test_m = test_m.T.to_dict().values()


In [156]:
type(train_m)
l = list(train_m)
type(l)
print(l[0])

{'language': 'en', 'signup_flow': 0, 'first_browser': 'Chrome', 'affiliate_channel': 'direct', 'counts': nan, 'account_active_lag': 'NB', 'affiliate_provider': 'direct', 'population_in_thousands': 'NB', 'first_device_type': 'Mac Desktop', 'signup_method': 'facebook', 'first_book_lag': 'NB', 'sum_secs_elapsed': nan, 'account_created_lag': 'greater 1 day', 'signup_app': 'Web', 'bookings': 'NB', 'first_affiliate_tracked': 'untracked', 'age': '0-4', 'gender': 'nan'}


In [157]:
vec = DictVectorizer()
train_m = vec.fit_transform(train_m)
test_m = vec.fit_transform(test_m)

In [158]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(target)
train_y = le.transform(target)
print (type(train_y))

<class 'numpy.ndarray'>


In [159]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import numpy as np
params = {}
params["objective"] = "multi:softmax"
params["num_class"] = 12
params["eta"] = 0.005
params["min_child_weight"] = 6
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 1
params["silent"] = 1
params["max_depth"] = 6
params['eval_metric'] = "ndcg@5"
params['nthread'] = 4

clf = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)  

plst = list(params.items())

dtrain = xgb.DMatrix(train_m, label=train_y)
dtest = xgb.DMatrix(test_m)
num_round = 5
model = xgb.train(plst, dtrain, num_round)
clf.fit(train_m, train_y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=0, learning_rate=0.3, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=25, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.5)

In [163]:
print(clf.feature_importances_)

[  2.36094445e-02   7.00280117e-03   1.16046416e-02   6.40256098e-03
   2.00080030e-04   4.00160061e-04   0.00000000e+00   6.00240077e-04
   2.20088032e-03   5.40216081e-03   5.20208105e-03   1.80072023e-03
   6.00240100e-03   7.00280117e-03   2.40096031e-03   0.00000000e+00
   2.80112051e-03   3.80152068e-03   0.00000000e+00   2.80112051e-03
   4.00160061e-04   3.00120050e-03   1.20048015e-03   6.00240100e-03
   2.00080030e-04   2.00080030e-04   0.00000000e+00   1.80072023e-03
   8.00320122e-04   2.00080030e-04   0.00000000e+00   4.00160061e-04
   0.00000000e+00   1.44057618e-02   3.20128049e-03   5.40216081e-03
   6.00240100e-03   8.80352128e-03   1.18047222e-02   6.20248122e-03
   8.80352128e-03   5.80232078e-03   0.00000000e+00   7.80312112e-03
   5.60224103e-03   6.60264120e-03   5.80232078e-03   1.14045618e-02
   6.60264120e-03   4.60184086e-03   3.80152068e-03   4.00160067e-03
   1.20048015e-03   1.32052824e-02   6.40256098e-03   3.60144046e-03
   1.34253696e-01   3.06122452e-02

In [160]:
print(dtest.feature_names)

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134']


In [161]:
print(train_m.shape)

(213451, 171)


In [162]:
#pred = model.predict(dtest)
pred = clf.predict(test_m)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [ ]:
pred = map(int,pred)

In [ ]:
pred = le.inverse_transform(pred)

In [ ]:
print set(pred)

In [ ]:
pred

In [ ]:
model.get_fscore()